In [1]:
# loading necessary packages
import cobra
from cobra import Model, Reaction, Metabolite
from cobra.io import load_model
from cobra.io import read_sbml_model
from cobra.medium import minimal_medium
import pandas as pd
import copy

In [12]:
model_path = "/projectnb2/talbot-lab-data/zrwerbin/soil_microbe_GEMs/iAA1300/iAA1300_modified_cobra.xml"
modelID = "iAA1300/iAA1300_modified_cobra.xml"
project_dir = "/projectnb2/talbot-lab-data/zrwerbin/soil_microbe_GEMs/"



In [10]:
major_carbon_sources={"MNXM726888_e":0, "MNXM1105810_e":0, "MNXM1137670_e":0, "MNXM1106000_e":0, "MNXM1104432_e":0, "MNXM1108175_e":0}
minor_carbon_sources={"MNXM726888_e":0, "MNXM738068_e":0, "MNXM1105810_e":0, "MNXM1137670_e":0, "MNXM1107769_e":0, "MNXM1106000_e":0, "MNXM1105732_e":0, "MNXM739527_e":0, "MNXM1107821_e":0, "MNXM1108206_e":0, "MNXM37_e":0, "MNXM741173_e":0, "MNXM29_e":0, "MNXM728337_e":0, "MNXM1106762_e":0, "MNXM1106164_e":0, "MNXM738804_e":0, "MNXM741664_e":0, "MNXM114_e":0, "MNXM737787_e":0, "MNXM142_e":0, "MNXM741553_e":0, "MNXM76_e":0, "MNXM199_e":0, "MNXM1104432_e":0, "MNXM1108175_e":0}
M8_media={"MNXM7454_e":1000, "MNXM421_e":1000, "MNXM26_e":1000, "MNXM168_e":1000, "MNXM1105732_e":1000, "MNXM739527_e":1000, "MNXM727871_e":1000, "MNXM1107821_e":1000, "MNXM1108206_e":1000, "MNXM304_e":1000, "MNXM458_e":1000, "MNXM128_e":1000, "MNXM1104756_e":1000, "MNXM1104738_e":1000, "MNXM726888_e":1000, "MNXM1107753_e":1000, "MNXM735978_e":1000, "MNXM90960_e":1000, "MNXM731166_e":1000, "MNXM738068_e":1000, "MNXM726711_e":1000, "MNXM726712_e":1000, "MNXM617_e":1000, "MNXM1105810_e":1000, "MNXM1137670_e":1000, "MNXM37_e":1000, "MNXM741173_e":1000, "MNXM29_e":1000, "MNXM490899_e":1000, "MNXM733114_e":1000, "MNXM1_e":1000, "WATER_e":1000, "MNXM60_e":1000, "MNXM1107769_e":1000, "MNXM728337_e":1000, "MNXM1105941_e":1000, "MNXM95_e":1000, "MNXM1104432_e":1000, "MNXM1106762_e":1000, "MNXM729132_e":1000, "MNXM1106164_e":1000, "MNXM1106000_e":1000, "MNXM738804_e":1000, "MNXM653_e":1000, "MNXM2255_e":1000, "MNXM2397_e":1000, "MNXM726092_e":1000, "MNXM27_e":1000, "MNXM274_e":1000, "MNXM8_e":1000, "MNXM729302_e":1000, "MNXM731950_e":1000, "MNXM732398_e":1000, "MNXM741664_e":1000, "MNXM249_e":1000, "MNXM9_e":1000, "MNXM734864_e":1000, "MNXM1094052_e":1000, "MNXM114_e":1000, "MNXM548_e":1000, "MNXM419_e":1000, "MNXM270_e":1000, "MNXM737787_e":1000, "MNXM1157_e":1000, "MNXM58_e":1000, "MNXM730135_e":1000, "MNXM142_e":1000, "MNXM741553_e":1000, "MNXM741004_e":1000, "MNXM76_e":1000, "MNXM158_e":1000, "MNXM199_e":1000, "MNXM174_e":1000, "MNXM729215_e":1000}


In [20]:
#model.summary()
model.compartments

{'c': '', 'e': '', 'p': '', 'C_e': ''}

In [25]:
default_growth_rate_list = []

output_major_carbon_sources = pd.DataFrame(major_carbon_sources, index=[0])
output_major_carbon_sources = output_major_carbon_sources.transpose()

output_minor_carbon_sources = pd.DataFrame(minor_carbon_sources, index=[0])
output_minor_carbon_sources = output_minor_carbon_sources.transpose()

no_c_growth_major_list = []
no_c_growth_minor_list = []

model_list = [modelID]

for modelID in model_list:
    
    model_path = project_dir + modelID
    print(model_path)
    model = read_sbml_model(model_path)
    ## Create no-carbon medium
    print(modelID)
    print(model_path)
    no_c_medium = model.medium # use full model media as template to create no-C
    print(no_c_medium)


    specific_media = M8_media.copy()

    # Set media-specific metabolites to 1000
    for met in specific_media:
        if met not in model.metabolites:
            #print("adding metabolite:")
            #print(met)
            model.add_metabolites([Metabolite(
                    met,
                    compartment ='e'),])
        
        if f"EX_{met}" not in model.reactions:  
            print(met)
            model.add_boundary(model.metabolites.get_by_id(met), type = "exchange")
            model.repair() # mentioned here: https://cobrapy.readthedocs.io/en/latest/faq.html
        no_c_medium[f"EX_{met}"] = 1000
    
    model.medium = no_c_medium
    print("default growth rate:")
    default_growth = model.slim_optimize()
    print(default_growth)
    default_growth_rate_list.append(default_growth)
    
    ## LOOP THROUGH MAJOR CARBON SOURCES    
    
    # Ensure carbon sources are all zero (even if they were in media)
    for met in major_carbon_sources:
        if met not in model.metabolites:
            #print("adding metabolite:")
            #print(met)
            model.add_metabolites([Metabolite(
                    met,
                    compartment ='C_e'),])
        
        if f"EX_{met}" not in model.reactions: 
            model.add_boundary(model.metabolites.get_by_id(met), type = "exchange")
            model.repair() # mentioned here: https://cobrapy.readthedocs.io/en/latest/faq.html
        no_c_medium[f"EX_{met}"] = 0
    
    # Confirm that the model will not grow once all these are set to zero
    
    print(no_c_medium)
    model.medium = no_c_medium
    print("No carbon source growth (control growth value):")
    no_c_growth = model.slim_optimize()
    no_c_growth_major_list.append(no_c_growth)
        
    print(no_c_growth) 
    print(model.summary())

    ## Loop through C sources
    # running the FBA with one C source at a time
    c_source_list = []
    for met in major_carbon_sources:
        print(met)
        with model:
            medium = model.medium
            medium[f"EX_{met}"] = 1000
            model.medium = medium
            growth = model.slim_optimize()
            print(growth)
            c_source_list.append(growth)
    
    output_major_carbon_sources[modelID] = c_source_list
    
    
    
    ## LOOP THROUGH MINOR CARBON SOURCES    
    # Ensure carbon sources are all zero (even if they were in media)
    for met in minor_carbon_sources:
        if met not in model.metabolites:
            #print("adding metabolite:")
            #print(met)
            model.add_metabolites([Metabolite(
                    met,
                    compartment ='C_e'),])
        
        if f"EX_{met}" not in model.reactions:  
            #model.add_boundary(met)
            model.add_boundary(model.metabolites.get_by_id(met), type = "exchange")
            model.repair() # mentioned here: https://cobrapy.readthedocs.io/en/latest/faq.html
        no_c_medium[f"EX_{met}"] = 0
    
    # Confirm that the model will not grow once all these are set to zero
    #print(no_c_medium)
    model.medium = no_c_medium
    
    print("No carbon source growth:")
    no_c_growth = model.slim_optimize()
    no_c_growth_minor_list.append(no_c_growth)
    
    c_source_list = []
    #c_source_list.append(no_c_growth)
    
    print(no_c_growth) 
    print(model.summary())
    
    
    ## Loop through C sources
    # running the FBA with one C source at a time
    for met in minor_carbon_sources:
        print(met)
        with model:
            medium = model.medium
            medium[f"EX_{met}"] = 1000
            model.medium = medium
            growth = model.slim_optimize()
            print(growth)
            c_source_list.append(growth)
    output_minor_carbon_sources[modelID] = c_source_list  


/projectnb2/talbot-lab-data/zrwerbin/soil_microbe_GEMs/iAA1300/iAA1300_modified_cobra.xml
iAA1300/iAA1300_modified_cobra.xml
/projectnb2/talbot-lab-data/zrwerbin/soil_microbe_GEMs/iAA1300/iAA1300_modified_cobra.xml
{'EX_MNXM9_e': 1000.0, 'EX_MNXM1_e': 1000.0, 'EX_MNXM726712_e': 0.05, 'EX_MNXM2255_e': 1000.0, 'EX_MNXM726711_e': 0.05, 'EX_MNXM1137670_e': 5.0, 'EX_MNXM729215_e': 1000.0, 'EX_MNXM653_e': 1000.0, 'EX_MNXM128_e': 1000.0, 'EX_MNXM731950_e': 1000.0, 'EX_MNXM731166_e': 1000.0, 'EX_MNXM90960_e': 1000.0, 'EX_WATER_e': 1000.0, 'EX_nh4_e': 5.0, 'EX_MNXM726092_e': 1000.0, 'EX_MNXM58_e': 1000.0, 'EX_MNXM95_e': 1000.0, 'EX_MNXM735438_e': 1000.0, 'EX_MNXM735978_e': 1000.0, 'EX_MNXM739758_e': 1000.0}
MNXM7454_e
MNXM421_e
MNXM26_e
MNXM168_e
MNXM1105732_e
MNXM739527_e
MNXM727871_e
MNXM1107821_e
MNXM1108206_e
MNXM304_e
MNXM458_e
MNXM1104756_e
MNXM1104738_e
MNXM726888_e
MNXM1107753_e
MNXM738068_e
MNXM617_e
MNXM1105810_e
MNXM37_e
MNXM741173_e
MNXM29_e
MNXM490899_e
MNXM733114_e
MNXM60_e
MNXM11

In [24]:
    print(model.summary())


Objective
1.0 BIOMASS_Av_DJ_core = 25.26944836056019

Uptake
------
   Metabolite         Reaction      Flux  C-Number C-Flux
MNXM1106164_e EX_MNXM1106164_e      3.54         6  0.16%
MNXM1107753_e EX_MNXM1107753_e     100.4         6  4.42%
MNXM1107769_e EX_MNXM1107769_e      2.95         6  0.13%
MNXM1107821_e EX_MNXM1107821_e     3.198         4  0.09%
    MNXM128_e     EX_MNXM128_e    0.1315         0  0.00%
    MNXM142_e     EX_MNXM142_e     11.04         4  0.32%
    MNXM158_e     EX_MNXM158_e     8.359         4  0.25%
      MNXM1_e       EX_MNXM1_e     50.62         0  0.00%
   MNXM2255_e    EX_MNXM2255_e   0.01746         0  0.00%
    MNXM274_e     EX_MNXM274_e   0.05756         6  0.00%
    MNXM304_e     EX_MNXM304_e 5.054E-05        10  0.00%
     MNXM37_e      EX_MNXM37_e     560.2         5 20.54%
 MNXM490899_e  EX_MNXM490899_e      1000        10 73.32%
     MNXM58_e      EX_MNXM58_e    0.1096         0  0.00%
    MNXM653_e     EX_MNXM653_e    0.2192         0  0.00%
 MNX

In [26]:
default_growth_rate = pd.DataFrame(default_growth_rate_list)
model = pd.DataFrame(model_list)

default_growth_rate_df = pd.concat([model, default_growth_rate], axis = 1)
default_growth_rate_df = default_growth_rate_df.transpose()
default_growth_rate_df.columns = default_growth_rate_df.iloc[0]
output_default_growth_rate = default_growth_rate_df.iloc[pd.RangeIndex(len(default_growth_rate_df)).drop(0)]
print(output_default_growth_rate)

0 iAA1300/iAA1300_modified_cobra.xml
0                          31.591408


In [27]:
print(output_major_carbon_sources)
#print(output_minor_carbon_sources)

               0  iAA1300/iAA1300_modified_cobra.xml
MNXM726888_e   0                           25.269448
MNXM1105810_e  0                           31.297825
MNXM1137670_e  0                           31.591408
MNXM1106000_e  0                           31.591408
MNXM1104432_e  0                           25.269448
MNXM1108175_e  0                           31.591408


In [28]:
no_c_growth_major = pd.DataFrame(no_c_growth_major_list)
model = pd.DataFrame(model_list)

no_c_growth_major_df = pd.concat([model, no_c_growth_major], axis = 1)
no_c_growth_major_df = no_c_growth_major_df.transpose()
no_c_growth_major_df.columns = no_c_growth_major_df.iloc[0]
output_no_c_growth_major = no_c_growth_major_df.iloc[pd.RangeIndex(len(no_c_growth_major_df)).drop(0)]
print(output_no_c_growth_major)

0 iAA1300/iAA1300_modified_cobra.xml
0                          25.269448


In [29]:
no_c_growth_minor = pd.DataFrame(no_c_growth_minor_list)
model = pd.DataFrame(model_list)

no_c_growth_minor_df = pd.concat([model, no_c_growth_minor], axis = 1)
no_c_growth_minor_df = no_c_growth_minor_df.transpose()
no_c_growth_minor_df.columns = no_c_growth_minor_df.iloc[0]
output_no_c_growth_minor = no_c_growth_minor_df.iloc[pd.RangeIndex(len(no_c_growth_minor_df)).drop(0)]
print(output_no_c_growth_minor)

0 iAA1300/iAA1300_modified_cobra.xml
0                          21.586503


In [30]:
output_major_carbon_sources = pd.concat([output_no_c_growth_major, output_major_carbon_sources], axis = 0)
output_minor_carbon_sources = pd.concat([output_no_c_growth_minor, output_minor_carbon_sources], axis = 0)

print(output_major_carbon_sources)
print(output_minor_carbon_sources)

              iAA1300/iAA1300_modified_cobra.xml    0
0                                      25.269448  NaN
MNXM726888_e                           25.269448  0.0
MNXM1105810_e                          31.297825  0.0
MNXM1137670_e                          31.591408  0.0
MNXM1106000_e                          31.591408  0.0
MNXM1104432_e                          25.269448  0.0
MNXM1108175_e                          31.591408  0.0
              iAA1300/iAA1300_modified_cobra.xml    0
0                                      21.586503  NaN
MNXM726888_e                           21.586503  0.0
MNXM738068_e                           21.586503  0.0
MNXM1105810_e                          27.363196  0.0
MNXM1137670_e                          27.633836  0.0
MNXM1107769_e                          21.834986  0.0
MNXM1106000_e                          27.633836  0.0
MNXM1105732_e                          21.586503  0.0
MNXM739527_e                           21.586503  0.0
MNXM1107821_e               

In [ ]:
# reading out C source loop output as a csv file for Gram negative M8 (complete) media gap-filled models
output_major_carbon_sources.to_csv(path_or_buf = project_dir + "output_major_carbon_sources_M8.csv", 
                                   sep = ",")
output_minor_carbon_sources.to_csv(path_or_buf = project_dir + "output_minor_carbon_sources_M8.csv", 
                                   sep = ",")

In [31]:
import memote

ModuleNotFoundError: No module named 'memote'